In [1]:
import numpy as np
import random
import math

In [2]:
def LineGenerate(lenght, m):
    """
    Creating a line with m random segments
    """
    line = np.zeros(lenght,)
    ab = np.array(sorted(random.sample(range(0, lenght-1), m*2))) # random start and end of the segments
    ab = ab.reshape((m,2)) # start and end of the segments coordinates
    # positive points coordinates
    ones = np.arange(ab[0,0],ab[0,1])
    for i in range(1,m):
        ones = np.concatenate((ones, np.arange(ab[i,0],ab[i,1])))
    line[ones] = 1  # put the generated segments on the line
    return line, ones, ab

In [3]:
def FindAssociated(c, ab, base, L):
    # distance from the farthest point to the base
    b = math.sqrt(c**2+base[1]**2)
    # distance from the farthest point to greedy
    dist = (L**2-2*L*b)/(2*(L-b-c))
    c_ass = c-dist
    for i in range(0,ab.shape[0]):
        if (c_ass > ab[i-1,1]) & (c_ass < ab[i,0]):
            c_ass = ab[i,0]
        if (c_ass < ab[0,0]):
            c_ass = ab[0,0]  
    return c_ass


In [4]:
def Candidates(base, L, ab):
    on_gap = False
    xB = ab[ab.shape[0]-1,1]
    c = np.array(xB)
    while not on_gap:
        # distance from the farthest point to the base
        b = math.sqrt(xB**2+base[1]**2)
        # distance from the farthest point to greedy
        dist = (L**2-2*L*b)/(2*(L-b-xB))
        # greedy point coordinate
        xA = xB-dist
        c = np.append(c, xA)
        xB = xA
        for i in range(0,ab.shape[0]):
            if (xA > ab[i-1,1]) & (xA < ab[i,0]):
                on_gap = True 
            if (xA < ab[0,0]):
                on_gap = True 
    c = c[:c.size-1]
    return c

In [5]:
def AllCandidates(base, L, ab):
    ab_cut = ab
    c = np.empty(0)
    while not (ab_cut.size == 0):
        c_i = Candidates(base, L, ab_cut)
        c = np.append(c, c_i)       
        ab_cut = ab_cut[:ab_cut.shape[0]-1]
    return c

In [6]:
def SegmentNumber(c, ab):
    for i in range(0,ab.shape[0]):
        if (c >= ab[i,0]) & (c <= ab[i,1]):
            return i  

In [7]:
def TourLenght(point1, point2, base):
    return max(point1, point2) - min(point1, point2) + math.sqrt(point1**2+base[1]**2) + math.sqrt(point2**2+base[1]**2)

In [8]:
def SeparateLeftRight(ab, base):
    ab_right = ab[np.argmax(ab[:,0] > base[0]):] - base[0]
    ab_left = np.flip(base[0] - ab[:np.argmax(ab[:,1] > base[0])])
    for i in range(0,ab.shape[0]):
        if (base[0] > ab[i,0]) & (base[0] < ab[i,1]):
            central_right = np.array([[base[0], ab[i,1]]])-base[0]
            central_left = -(np.array([[base[0],ab[i,0]]])-base[0])
            ab_right = np.append(central_right, ab_right, axis=0)
            ab_left = np.append(central_left, ab_left, axis=0)
    return ab_left, ab_right

In [9]:
def FindCentralTours(c_left, c_right , base, L):
    j = 0
    central_tours = np.empty((0,3))
    tour_lenght = 0
    while (tour_lenght <= L) and (j < c_left.size):
        central_tours_1 = np.empty((0,3))
        i = 0
        while (tour_lenght <= L) and (i < c_right.size):
            dist_left = math.sqrt(c_left[j]**2+base[1]**2)
            dist_right = math.sqrt(c_right[i]**2+base[1]**2)
            tour_lenght = c_left[j] + c_right[i] + dist_left + dist_right
            tour = np.array((c_left[j], c_right[i], tour_lenght)).reshape(1,3)
            central_tours_1 = np.append(central_tours_1, tour, axis=0)
            i += 1
        central_tours = np.append(central_tours, central_tours_1[central_tours_1[:,2]<=L,:], axis=0)
        tour_lenght = 0
        j += 1
    return central_tours

In [10]:
def DPwithPoints(c, ab, base, L):   
    c = np.sort(c)  
    E = np.zeros(c.size) # check if zeros are ok
    Start_Points = np.zeros((c.size,c.size)) # check if zeros are ok
    End_Points = np.zeros((c.size,c.size))

    for k in range(0, c.size):
        c_ass = FindAssociated(c[k], ab, base, L)
        
        j = np.arange(SegmentNumber(c_ass, ab), SegmentNumber(c[k], ab)+1)  
    
        if c_ass == ab[0,0]:
            E[k] = TourLenght(c[k], ab[0,0], base)  
            
            Start_Points[k,0] = c[k]
            End_Points[k,0] = ab[0,0]
            
        elif c_ass in ab[1:,0]:      
            
            func = np.empty(0)
            for i in j:
                func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])
            E[k] = min(func)
            index_min = np.argmin(func)+j[0]
            
            Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
            End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                                   
            Start_Points[k, np.argmax(Start_Points[k,:]==0)] = c[k]
            End_Points[k, np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
    
        else:             
            func = np.empty(0)
            if (c_ass < ab[SegmentNumber(c[k], ab),0]): 
                for i in j[1:]:
                    func = np.append(func, TourLenght(c[k], ab[i,0], base) + E[c.tolist().index(ab[i-1,1])])  
            else:
                func = [1000000.0]
            func2 = L + E[c.tolist().index(c_ass)]
            
            if (func2>min(func)):
                index_min = np.argmin(func)+j[0]
                Start_Points[k,:] = Start_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,:] = End_Points[c.tolist().index(ab[index_min-1,1]),:]
                End_Points[k,np.argmax(End_Points[k,:]==0)] = ab[index_min,0]
            else:
                Start_Points[k,:] = Start_Points[c.tolist().index(c_ass),:] 
                End_Points[k,np.argmax(End_Points[k,:]==0)] = c_ass
                
            Start_Points[k,np.argmax(Start_Points[k,:]==0)] = c[k]
                                 
            E[k] = min(func2, min(func))
            
    Start_Points = Start_Points[c.size-1]
    End_Points = End_Points[c.size-1]
    return E[c.size-1], Start_Points[Start_Points>0], End_Points[End_Points>0]

In [11]:
def OneSideWithPoints(c, c_first, ab):
    E = np.zeros(c_first.size)
    StartPoints = np.zeros((c_first.size,c.size))
    EndPoints = np.zeros((c_first.size,c.size))
    ab_i = ab.astype(np.float64)
    for i in range(0, c_first.size):
        if c_first[i] != ab_i[0,1]:
            ab_i[0,0] = c_first[i]
        else:
            ab_i = ab[ab[:,0]>c_first[i]].astype(np.float64)
        if (c[c>c_first[i]].size != 0):
            E[i],  Start_Points, End_Points = DPwithPoints(c[c>c_first[i]], ab_i, base, L) 
            StartPoints[i,:Start_Points.size] = Start_Points
            EndPoints[i,:End_Points.size] = End_Points
            
        else: E[i] = 0
    return E, StartPoints, EndPoints

In [12]:
def JoinTours(StartPoints, EndPoints):
    Start = np.array([StartPoints[StartPoints>0]])
    End = np.array([EndPoints[EndPoints>0]])
    
    return np.append(Start, End, axis = 0)

In [13]:
def BothSidesWithPoints(ab, base, L):
    ab_left, ab_right = SeparateLeftRight(ab, base)
    
    c_right = np.sort(AllCandidates(base, L, ab_right))
    c_left =  np.sort(AllCandidates(base, L, ab_left))
    
    central_tours = FindCentralTours(c_left, c_right , base, L)
    
    if not (central_tours.size == 0):  
        c_left_first = np.unique(np.sort(central_tours[:,0]))
        c_right_first = np.unique(np.sort(central_tours[:,1]))   
        
        E_left, StartPoints_left, EndPoints_left = OneSideWithPoints(c_left, c_left_first, ab_left)
        E_right, StartPoints_right, EndPoints_right = OneSideWithPoints(c_right, c_right_first, ab_right)
        
        left_central = np.append(c_left_first.reshape(E_left.shape[0],1), E_left.reshape(E_left.shape[0],1), axis=1)
        right_central = np.append(c_right_first.reshape(E_right.shape[0],1), E_right.reshape(E_right.shape[0],1), axis=1)
      
        totalL = np.zeros(central_tours.shape[0])
        for i in range (0, central_tours.shape[0]):
            a = tuple(np.argwhere(left_central == central_tours[i,0])[0])
            b = tuple(np.argwhere(right_central == central_tours[i,1])[0])
            totalL[i] = left_central[a[0],1] + right_central[b[0],1] + float(central_tours[i,2])
        
        c = tuple(np.argwhere(left_central == central_tours[np.argmin(totalL),0])[0])
        d = tuple(np.argwhere(right_central == central_tours[np.argmin(totalL),1])[0])
    else: 
        totalL = np.array([10000])
              

    E_left_max, StartPoints_left_max, EndPoints_left_max = DPwithPoints(c_left, ab_left, base, L)
    E_right_max, StartPoints_right_max, EndPoints_right_max = DPwithPoints(c_right, ab_right, base, L)
    
    print(StartPoints_right_max)
    print(EndPoints_right_max)
    
    if (E_left_max + E_right_max) < min(totalL):
        
        Tour_right = JoinTours(StartPoints_right_max, EndPoints_right_max) + base[0]
        #Tour_right = JoinTours(StartPoints_right_max, EndPoints_right_max) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left_max, EndPoints_left_max)
        #Tour_left = base[0] - JoinTours(StartPoints_left_max[0], EndPoints_left_max[0])
        Tour = np.append(Tour_left, Tour_right, axis = 1)
        
    else:       
        Tour_right = JoinTours(StartPoints_right[d[0]], EndPoints_right[d[0]]) + base[0]
        Tour_right = Tour_right[ ::-1, :]
        Tour_left = base[0] - JoinTours(StartPoints_left[c[0]], EndPoints_left[c[0]])
        Tour_central = np.array([[base[0] - central_tours[np.argmin(totalL),0], central_tours[np.argmin(totalL),1] + base[0]]]).reshape((2, 1))
        
        Tour = np.append(Tour_left, Tour_central, axis = 1)
        Tour = np.append(Tour, Tour_right, axis = 1)
        
    TotalLenght = min((E_left_max + E_right_max), min(totalL)) 
    
    for i in range(Tour.shape[1]):
        print('Tour', i+1, 'start:', Tour[0,i])
        print('Tour', i+1, 'end:', Tour[1,i])
        print('Tour', i+1, 'lenght:', TourLenght(abs(Tour[0,i]-base[0]), abs(Tour[1,i]-base[0]), base))
    print('Total number of tours:', Tour.shape[1])
    print('Total lenght:', TotalLenght)
    
    return Tour, TotalLenght

In [14]:
line, ones, ab = LineGenerate(400, 10)
base = np.array((200,500)) # base station coordinates
L = 1120

In [15]:
ab

array([[ 11,  54],
       [143, 170],
       [223, 260],
       [261, 275],
       [279, 284],
       [288, 296],
       [297, 303],
       [331, 338],
       [340, 348],
       [362, 391]])

In [16]:
Tour, TotalLenght = BothSidesWithPoints(ab, base, L)
Tour = np.transpose(Tour)
Tour = Tour - base[0]

[103. 191.]
[ 23. 131.]
Tour 1 start: 143.0
Tour 1 end: 170.0
Tour 1 lenght: 1031.137703494496
Tour 2 start: 11.0
Tour 2 end: 54.0
Tour 2 lenght: 1098.408789000933
Tour 3 start: 223.0
Tour 3 end: 303.0
Tour 3 lenght: 1091.0274961633008
Tour 4 start: 331.0
Tour 4 end: 391.0
Tour 4 lenght: 1112.1153929370519
Total number of tours: 4
Total lenght: 4332.689381595781


In [17]:
def FindTheFarthest(base,first,last):
    """
    Looking for the farthest point from the base
    """
    # distances to the base station
    fdist = math.sqrt(first**2+base[1]**2)
    ldist = math.sqrt(last**2+base[1]**2)

    # comparing the distances
    if fdist > ldist:
        farthest = first
        dist = fdist
    else:
        farthest = last
        dist = ldist
    
    return farthest, dist

In [18]:
def TourLength(tour):
    dist_left = math.sqrt(tour[0]**2+base[1]**2)
    dist_right = math.sqrt(tour[1]**2+base[1]**2)
    tour_lenght = abs(tour[0]-tour[1]) + dist_left + dist_right
    return tour_lenght

In [20]:
def OneToEachDisribution(Tour):
    Tour1 = np.empty(0)
    Tour2 = np.empty(0)
    while Tour.size>1:
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            Tour1 = np.append(Tour1, Tour[0])
            Tour = Tour[1:]
        else:
            Tour1 = np.append(Tour1, Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]
        
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            Tour2 = np.append(Tour2, Tour[0])
            Tour = Tour[1:]
        else:
            Tour2 = np.append(Tour2, Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]  
            
    return Tour1, Tour2

In [19]:
def ToTheMinDisribution(Tour):
    Tour1 = np.empty(0)
    Tour2 = np.empty(0)
    M1 = 0
    M2 = 0
    while Tour.size>1:
        farthest, dist = FindTheFarthest(base,Tour[0,0],Tour[Tour.shape[0]-1,1])
        if farthest == Tour[0,0]:
            if M1 < M2:
                Tour1 = np.append(Tour1, Tour[0])
                M1 += TourLength(Tour[0])
            else:
                Tour2 = np.append(Tour2, Tour[0])
                M2 += TourLength(Tour[0])
            Tour = Tour[1:]
        else:
            if M1 < M2:
                Tour1 = np.append(Tour1, Tour[Tour.shape[0]-1])
                M1 += TourLength(Tour[Tour.shape[0]-1])
            else:
                Tour2 = np.append(Tour2, Tour[Tour.shape[0]-1])
                M2 += TourLength(Tour[Tour.shape[0]-1])
            Tour = Tour[:Tour.shape[0]-1]
    Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)
    Tour2 = Tour2.reshape(Tour2.shape[0]//2,2)
    if M1 > M2:
        Tour1, Tour2 = Tour2, Tour1
        M1, M2 = M2, M1

    print("T1 =", Tour1)
    print("T1 total length: M1 =", M1)
    print("T2 =",Tour2)
    print("T2 total length: M2 =", M2)   
    return Tour1, Tour2, M1, M2

In [21]:
def GetSubTour1(Tour, Tour1): 
    T11 = np.empty(0)
    for tour in Tour1:
        if TourLength(tour)<L:
            T11 = np.append(T11, tour)
    T11 = T11.reshape(T11.shape[0]//2,2)

    T1 = np.empty(0)
    
    for tour in T11:
        a = np.where(Tour == tour)[0][0]
        if a < Tour.shape[0]-1:
            if (TourLength(Tour[a+1]) >= L) and (Tour[a,1] == Tour[a+1,0]):
                T1_right = np.append(T1_right, tour)
                
            if (TourLength(Tour[a-1]) >= L) and (Tour[a,0] == Tour[a-1,0]):
                T1_left = np.append(T1_left, tour)
    
    return T1_right.reshape(T1_right.shape[0]//2,2),  T1_left.reshape(T1_left.shape[0]//2,2)

In [22]:
def GetSubTour2(Tour2, base):    
    T2 = np.empty(0)
    for tour in Tour2:
        dist_left = math.sqrt(tour[0]**2+base[1]**2)
        dist_right = math.sqrt(tour[1]**2+base[1]**2)
        if (dist_left + dist_right)<l:
            T2 = np.append(T2, tour)
    return T2.reshape(T2.shape[0]//2,2)

In [23]:
def FindXR(tour, base,l):
    if abs(tour[1]) > abs(tour[0]):
        closest = abs(tour[0])
    else:
        closest = abs(tour[1])
    
    a = math.sqrt(closest**2+base[1]**2)
    x = (l-a*2)/2
    d = math.sqrt((closest+x)**2+base[1]**2)
    r = d - a - x
    return x, r

In [24]:
def FindXS(tour, base,l):
    if abs(tour[1]) > abs(tour[0]):
        farthest = abs(tour[1])
    else:
        farthest = abs(tour[0])  
    x = l/2
    c = math.sqrt(farthest**2+base[1]**2)
    d = math.sqrt((farthest+x)**2+base[1]**2)
    s = d - c - x
    return x, s

In [25]:
"STEP 2"
Tour = np.sort(Tour, axis = 0)
print('T1 = ', Tour)

T1 =  [[-189. -146.]
 [ -57.  -30.]
 [  23.  103.]
 [ 131.  191.]]


In [26]:
"STEP 3"
Tour1, Tour2, M1, M2 = ToTheMinDisribution(Tour)

T1 = [[131. 191.]
 [-57. -30.]]
T1 total length: M1 = 2143.253096431548
T2 = [[-189. -146.]
 [  23.  103.]]
T2 total length: M2 = 2189.4362851642336


In [27]:
"STEP 4"
l = M2 - M1
A = M1 + l/2

In [28]:
"STEP 5"
SubTour2 = GetSubTour2(Tour2, base)
"STEP 6"
SubTour1_right,  SubTour1_left= GetSubTour1(Tour, Tour1)

In [29]:
"STEP 7"
xr = np.empty(0)
r = np.empty(0)
for tour in SubTour2:    
    x_k, r_k = FindXR(tour, base, l)
    xr = np.append(xr, x_k)
    r = np.append(r, r_k)  
    
    print("t = ", tour)      
    print("xr = ", xr)    
    print("r = ", r)
    print("_____________")

In [30]:
"STEP 8"
xs = np.empty(0)
s = np.empty(0)
for tour in SubTour1_right:
    x_n, s_n = FindXS(tour, base, l)
    xs_right = np.append (xs,  x_n)
    s = np.append(s, s_n)

for tour in SubTour1_left:
    x_n, s_n = FindXS(tour, base, l)
    xs_left = np.append (xs,  x_n)
    s = np.append(s, s_n)

In [31]:
"STEP 9"
if (s.size > 0) and (s.size > 0):
    minrs = min(np.min(r), np.min(s))
else: 
    if r.size > 0:
        minrs = np.min(r)
    else:
        if s.size > 0:
            minrs = np.min(s)
        else:
            minrs = 0    

In [32]:
"STEP 10"
# We are going from the base one by one, right?
# Is it posible we need to devide not the closest? Should we compare?
if minrs != 0:
    if minrs == np.min(r):
        if abs(Tour2[Tour2.shape[0]-1, 1]) < abs(Tour2[Tour2.shape[0]-1, 0]):
            a = Tour2[Tour2.shape[0]-1, 1]
            Tour2[Tour2.shape[0]-1, 1] = Tour2[Tour2.shape[0]-1, 1] - xr[Tour2.shape[0]-1] 
            newTour = np.array([Tour2[Tour2.shape[0]-1, 1], a])        
        else:
            a = Tour2[Tour2.shape[0]-1, 0]
            Tour2[Tour2.shape[0]-1, 0] = Tour2[Tour2.shape[0]-1, 0] + xr[Tour2.shape[0]-1] 
            newTour = np.array([a, Tour2[Tour2.shape[0]-1, 0]])
        Tour1 = np.append(Tour1, newTour)
        
    if Tour1.shape[0]>1:
        Tour1 = Tour1.reshape(Tour1.shape[0]//2,2)

In [33]:
"STEP 11"
if minrs != 0:
    if minrs == np.min(s):
        
        Tour1[0,1] = Tour1[0,1]+xs_right[0]
        Tour2[0,0] = Tour2[0,0]+xs_right[0]
        
        

In [34]:
ab-base[0]

array([[-189, -146],
       [ -57,  -30],
       [  23,   60],
       [  61,   75],
       [  79,   84],
       [  88,   96],
       [  97,  103],
       [ 131,  138],
       [ 140,  148],
       [ 162,  191]])

In [35]:
Tour1

array([[131., 191.],
       [-57., -30.]])

In [36]:
Tour2

array([[-189., -146.],
       [  23.,  103.]])

In [37]:
Tour1Lenght = 0
Tour2Lenght = 0
for tour in Tour1:
    Tour1Lenght += TourLength(tour)
for tour in Tour2:
    Tour2Lenght += TourLength(tour)    

In [38]:
Tour1Lenght

2143.253096431548

In [39]:
Tour2Lenght

2189.4362851642336

In [40]:
# Questions
# STEP 10: Is it posible we need to devide not the closest? Should we compare?

In [41]:
# Cases:
# 5 Tours -> divide
# 4 Tours -> nothing
# 2 Tour -> enlarge

In [42]:
# TODO:
# STEP 11
# Universal case
# *Enlarge to the left

# DP
# Error with few tours (StartPoints_right_max, EndPoints_right_max) array sizes   

# Clear code and write better the algothm